In [ ]:
import os
import scipy
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import cv2 as cv

labels=['can', 'plastic']

In [ ]:
tf.test.is_gpu_available()

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),


    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),

    #tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    #tf.keras.layers.BatchNormalization(),
    #tf.keras.layers.MaxPooling2D(2, 2),
    #tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Flatten(),  
    tf.keras.layers.Dense(2, activation='softmax'),
])
model.summary()

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.1
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

train_generator = train_datagen.flow_from_directory(
    'C:/data/train',
    target_size=(256, 256),
    batch_size=16,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=0
)

validation_generator = test_datagen.flow_from_directory(
    'C:/data/train',
    target_size=(256, 256),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    shuffle=True,
    seed=0
)


checkpoint_path = 'C:/Users/jihun/Classification_using_cnn/model'

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('loss')<0.1):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training = True

    def modelcheckpoint(epoch, logs={}):
        ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,
                        save_best_only=True,
                        monitor='val_loss',
                        mode='min',
                        verbose=1)
        return checkpoint_callback

checkpoint = modelcheckpoint()
callbacks = myCallback()

In [ ]:
import tensorflow as tf
checkpoint_path = 'C:/Users/jihun/Classification_using_cnn/model/check.ckpt'

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                verbose=1,
                                                save_weight_only=True,
                                                save_freq=5)

In [ ]:
history = model.fit(train_generator, epochs=150, verbose=1, validation_data=validation_generator)

In [ ]:
model.save('C:/Users/jihun/Classification_using_cnn/model/my_model_img316_data4000_epoch150.h5')

In [ ]:
model.save(modelcheckpoint())

In [ ]:
avg_ac = history.history['accuracy']
avg=0
for i in range(len(avg_ac)):
    avg += avg_ac[i]
print(avg/10)

In [ ]:
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(train_acc) + 1)

plt.plot(epochs, train_acc, 'b*-', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'r', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, train_loss, 'b*-', label = 'Training loss')
plt.plot(epochs, val_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
test_x, test_y = validation_generator.__getitem__(1)
# test_x is used to train the data
# test_y refers to the actual answer of the testing set data
# preds refers to the predicted class
preds = model.predict(test_x)

plt.figure(figsize=(20, 20))
for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.title('pred:%s / truth:%s' % (labels[np.argmax(preds[i])], labels[np.argmax(test_y[i])]))
    plt.imshow(test_x[i])

In [ ]:
print("-- Evaluate --")
scores = model.evaluate_generator(validation_generator, steps=20)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
history = model.fit(train_generator, epochs=300, verbose=1, validation_data=validation_generator)

In [ ]:
model.save('C:/Users/jihun/Classification_using_cnn/model/my_model_img316_data4000_epoch300.h5')

In [ ]:
print("-- Evaluate --")
scores = model.evaluate_generator(validation_generator, steps=20)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(train_acc) + 1)

plt.plot(epochs, train_acc, 'b*-', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'r', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, train_loss, 'b*-', label = 'Training loss')
plt.plot(epochs, val_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
test_x, test_y = validation_generator.__getitem__(1)
# test_x is used to train the data
# test_y refers to the actual answer of the testing set data
# preds refers to the predicted class
preds = model.predict(test_x)

plt.figure(figsize=(20, 20))
for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.title('pred:%s / truth:%s' % (labels[np.argmax(preds[i])], labels[np.argmax(test_y[i])]))
    plt.imshow(test_x[i])

In [ ]:
import os
test_path = 'C:/Users/jihun/Classification_using_cnn/test_img'
file = os.listdir(test_path)

In [ ]:
test_dir = 'C:/Users/jihun/Classification_using_cnn/test_img'  # 테스트 이미지 폴더 경로
target_size = (256, 256)  # 이미지 크기
test_class=[]
pred=[]
class_labels = {0: 'can', 1: 'plastic'}  # 클래스 레이블과 인덱스 매핑
i=0
for filename in os.listdir(test_dir):
    if filename.endswith('.jpg'):
        image_path = os.path.join(test_dir, filename)
        img = image.load_img(image_path, target_size=target_size)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0  # 이미지 정규화

        # 예측 수행
        predictions = model.predict(img_array)

        # 예측 결과 해석
        predicted_class_index = np.argmax(predictions[0])
        predicted_class_label = class_labels[predicted_class_index]

        # 실제 클래스 레이블
        actual_class_label = filename.split('.')[0]
        if actual_class_label[0] == 'c' :
            test_class.append(0)
        elif actual_class_label[0] == 'p':
            test_class.append(1)

        if predicted_class_label[0] == 'c':
            pred.append(0)
        elif predicted_class_label[0] =='p':
            pred.append(1)
      
        # 결과 표시
        print('이미지:', filename)
        print('실제 클래스:', actual_class_label)
        print('예측된 클래스:', predicted_class_label)
        print()

        # 이미지와 예측 결과 표시
        plt.imshow(img)
        plt.xticks([])
        plt.yticks([])
        plt.title('Actual: {}   Pred: {}'.format(actual_class_label, predicted_class_label))
        plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# test_class와 pred 리스트를 numpy 배열로 변환합니다.
test_class = np.array(test_class)
pred = np.array(pred)

# 정확도를 계산합니다.
accuracy = np.mean(test_class == pred)
print('정확도: {:.2%}'.format(accuracy))

# 혼동 행렬을 생성합니다.
unique_labels = np.unique(np.concatenate((test_class, pred)))
cm = np.zeros((len(unique_labels), len(unique_labels)), dtype=int)

for true_label, predicted_label in zip(test_class, pred):
    cm[true_label][predicted_label] += 1

# 혼동 행렬을 출력합니다.
print('혼동 행렬:')
print(cm)

# 혼동 행렬을 시각화합니다.
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)

# 사각형 안에 숫자와 레이블을 표시합니다.
thresh = cm.max() / 2.
for i in range(len(unique_labels)):
    for j in range(len(unique_labels)):
        plt.text(j, i, cm[i, j], ha='center', va='center', color='white' if cm[i, j] > thresh else 'black')
        plt.text(j, i+0.2, 'Pred: {}'.format(unique_labels[j]), ha='center', va='center', color='black', fontsize=8)
        plt.text(j, i-0.2, 'True: {}'.format(unique_labels[i]), ha='center', va='center', color='black', fontsize=8)

plt.title('confusion matrix')
plt.colorbar()
tick_marks = np.arange(len(unique_labels))
plt.xticks(tick_marks, unique_labels)
plt.yticks(tick_marks, unique_labels)
plt.xlabel('predict')
plt.ylabel('true')
plt.tight_layout()
plt.show()